# Doing transfer learning with Keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-04-06 11:35:50.713899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Load in pre-trained parameters from imagenet and define model by modifying input layer for image size, and output layer for binary classification

In [2]:
base_model = keras.applications.Xception(
    weights="imagenet",
    input_shape=(600, 600, 3),  # images are 600x600, 3 colour channels
    include_top=False
    )

base_model.trainable = False  # do not update pretrained parameters

inputs = keras.Input(shape=(600, 600, 3))  # new input layer to be trained

x = base_model(inputs, training=False)  # define x as output from model

x = keras.layers.GlobalAveragePooling2D()(x)  # do pooling to get output for reasons

outputs = keras.layers.Dense(1)(x)  # output layer is binary classification
model = keras.Model(inputs, outputs)  # gives full model

2023-04-06 11:35:54.139330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Need to import dataset

In [3]:
## get tf.data.Dataset objects for each
training_data, validation_data = keras.utils.image_dataset_from_directory(
    "Pistachio_Image_Dataset",
    labels="inferred",
    label_mode="binary",
    color_mode="rgb",
    batch_size=32,
    image_size=(600, 600),
    seed=0,
    validation_split=0.2,
    subset="both",
)

Found 2148 files belonging to 2 classes.
Using 1719 files for training.
Using 429 files for validation.


Now can train the model

In [4]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.fit(training_data, epochs=10, validation_data=validation_data)

Epoch 1/10
54/54 [==============================] - 1608s 30s/step - loss: 8.6885 - binary_accuracy: 0.4270 - val_loss: 8.5310 - val_binary_accuracy: 0.4406
Epoch 2/10
54/54 [==============================] - 1724s 32s/step - loss: 8.8000 - binary_accuracy: 0.4229 - val_loss: 8.5310 - val_binary_accuracy: 0.4406
Epoch 3/10
54/54 [==============================] - 1642s 31s/step - loss: 8.8000 - binary_accuracy: 0.4229 - val_loss: 8.5310 - val_binary_accuracy: 0.4406
Epoch 4/10
36/54 [===================>..........] - ETA: 7:23 - loss: 8.7630 - binary_accuracy: 0.4253

KeyboardInterrupt: 

This would take hours to run on my shitty laptop so I'm cutting it off partway through